### Railway PostgreSQL DB Connection

This file is used to import the data and fill in the SQL DB hosted on RailwayApp.
  

#### Step 1 - Create Tables

In [2]:

import pandas as pd
import os
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv

In [2]:
# Code below includes relevant module as well as connection with the BTC News function so that it is triggered and sends the details into the 

import pandas as pd
import os
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv


load_dotenv()


db_url = os.getenv('db_url')
conn = psycopg2.connect(db_url)

cur = conn.cursor()

cur.execute('''
    CREATE TABLE IF NOT EXISTS btc_news (
        Date DATE,
        Title TEXT PRIMARY KEY
)
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS btc_prices (
        date DATE PRIMARY KEY,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume FLOAT
)        
''')

conn.commit()

conn.close()

           Date                                              Title
0    2024-05-30     European bitcoin ETPs suffer mounting outflows
1    2024-05-24  British-Chinese bitcoin money launderer jailed...
2    2024-05-24                      Cryptofinance: into the ether
3    2024-05-23  SEC paves way for ethereum ETFs in boost for c...
4    2024-05-22           First UK crypto ETPs to launch on May 28
..          ...                                                ...
100  2023-12-07                               The return of crypto
101  2023-12-05                       Bitcoin’s bounceback déjà vu
102  2023-12-05                     The jobs market is still tight
103  2023-12-05  Buying frenzy puts some Grayscale crypto funds...
104  2023-12-04  Bitcoin price surges above $42,000 as rate cut...

[105 rows x 2 columns]


#### Step 2 - Fill in tables

In [10]:
# function to send data into the postgresql db

from fun_btc_news import scrape_ft_bitcoin_articles

def insert_btc_news(conn, data):
    try:
        cursor = conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO btc_news(date, title)
                VALUES (%s, %s)
                ON CONFLICT (title) DO NOTHING
            ''', (row['Date'], row['Title']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f'Error inserting btc news data: {e}')
        conn.rollback()

In [11]:
# function to send data into the postgresql db

from fun_btc_price import get_crypto_data

def insert_btc_prices(conn, data):
    try:
        cursor= conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO btc_prices(date, open, high, low, close, volume)
                VALUES(%s, %s, %s, %s, %s, %s)
                ON CONFLICT (date) DO NOTHING
            ''', (row['date'], row['open'], row['high'], row['low'], row['close'], row['volume']))
        conn.commit()
        conn.close()
    except Exception as e:
        print(f'Error inserting btc prices: {e}')
        conn.rollback()

#### Step 3 - Main function

This main function will serve as a "master" function that will trigger both fetching data and filling in the received data into the cloud db with a query. 

In [12]:
# calling the main/master function which will trigger both scraping and API fetching and will subsequently send the data to the db

def main():
    
    btc_data_news = scrape_ft_bitcoin_articles() # this triggers the "news" function and stores the result in the variable
    
    btc_data_prices = get_crypto_data(symbol='BTC', market='USD', past_months=6) # same as above but with prices
    
    conn = psycopg2.connect(db_url) # establish a connection with the cloud db
    
    if conn: # once data is obtained and connection is established, data is filled into the db
        insert_btc_news(conn, btc_data_news)
        insert_btc_prices(conn, btc_data_prices)
        
        conn.close()
    else:
        print('Failed to connect to the database')

if __name__ == '_main_':
    main()
        

In [14]:
main()

Error inserting btc news data: not all arguments converted during string formatting
Error inserting btc prices: not all arguments converted during string formatting


#### Step 4 - Code aggregation

Below code is consolidated into a continous script that can be used in an automated environment (for future testing with AWS lambda)

In [15]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from fun_btc_price import get_crypto_data
from fun_btc_news import scrape_ft_bitcoin_articles

load_dotenv()
db_url = os.getenv('db_url')

def insert_btc_news(conn, data):
    try:
        cursor = conn.cursor()
        for _, row in data.iterrows():
            cursor.execute('''
                INSERT INTO btc_news (date, title)
                VALUES (%s, %s)
                ON CONFLICT (title) DO NOTHING
            ''', (row['Date'], row['Title']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f'Error inserting btc news data: {e}')
        conn.rollback()

def insert_btc_prices(conn, data):
    try:
        cursor = conn.cursor()
        for _, row in data.iterrows():
            cursor.execute('''
                INSERT INTO btc_prices (date, open, high, low, close, volume)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (date) DO NOTHING
            ''', (row['date'], row['open'], row['high'], row['low'], row['close'], row['volume']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f'Error inserting btc prices data: {e}')
        conn.rollback()

def main():
    btc_data_news = scrape_ft_bitcoin_articles()
    btc_data_prices = get_crypto_data(symbol='BTC', market='USD', past_months=6)
    
    # Debug: Print DataFrames to ensure they contain data
    print("BTC News DataFrame:")
    print(btc_data_news.head())
    print("\nBTC Prices DataFrame:")
    print(btc_data_prices.head())
    
    conn = psycopg2.connect(db_url)
    
    if conn:
        print("Connected to the database.")
        insert_btc_news(conn, btc_data_news)
        insert_btc_prices(conn, btc_data_prices)
        conn.close()
        print("Data inserted and connection closed.")
    else:
        print('Failed to connect to the database')

if __name__ == "__main__":
    main()


BTC News DataFrame:
        Date                                              Title
0 2024-05-30     European bitcoin ETPs suffer mounting outflows
1 2024-05-24  British-Chinese bitcoin money launderer jailed...
2 2024-05-24                      Cryptofinance: into the ether
3 2024-05-23  SEC paves way for ethereum ETFs in boost for c...
4 2024-05-22           First UK crypto ETPs to launch on May 28

BTC Prices DataFrame:
        date      open      high       low     close        volume
0 2023-12-01  37732.27  39000.00  37618.25  38703.54  16395.901978
1 2023-12-02  38706.55  39740.00  38660.03  39471.48   7540.768818
2 2023-12-03  39467.42  40221.22  39293.02  39984.81   7153.156638
3 2023-12-04  39984.81  42410.46  39983.60  41987.24  21993.059956
4 2023-12-05  41985.53  45000.00  41384.03  44084.39  24018.760355
Connected to the database.
Data inserted and connection closed.


#### Step 5 - Query execution

Example queries to review the data that has been passed onto the cloud db.


In [4]:
# getting data from DB executing sql queries using a connector

import pandas as pd
import os
import psycopg2
from dotenv import load_dotenv

# Load database URL from .env file
load_dotenv()
db_url = os.getenv('db_url')

# Function to execute the query and fetch results
def fetch_news_with_prices_containing_ftc(conn):
    try:
        cursor = conn.cursor()
        # Define the SQL query
        query = '''
            SELECT 
                n.date, 
                n.title, 
                p.close, 
                p.volume
            FROM 
                btc_news n
            JOIN 
                btc_prices p ON n.date = p.date
            WHERE 
                n.title ILIKE '%SEC%'
        '''
        # Execute the query
        cursor.execute(query)
        # Fetch all results
        results = cursor.fetchall()
        # Define column names for the result set
        columns = ['date', 'title', 'close', 'volume']
        # Convert the results to a DataFrame
        df = pd.DataFrame(results, columns=columns)
        cursor.close()
        return df
    except Exception as e:
        print(f'Error executing query: {e}')
        return None

# Main function to connect to the database and run the query
def main():
    conn = psycopg2.connect(db_url)
    if conn:
        news_with_prices_df = fetch_news_with_prices_containing_ftc(conn)
        if news_with_prices_df is not None:
            print(news_with_prices_df)
        conn.close()
    else:
        print('Failed to connect to the database')

# Execute the main function
if __name__ == "__main__":
    main()

          date                                              title     close  \
0   2024-01-08  Price war for spot bitcoin ETF heats up ahead ...  46995.10   
1   2024-01-10  Bitcoin swings sharply after false claim that ...  46666.11   
2   2024-01-10        Fake bitcoin post shows SEC crypto quandary  46666.11   
3   2024-01-10  Bitcoin X hack an ‘embarrassment’ for security...  46666.11   
4   2024-01-11  SEC approves first spot bitcoin ETFs in boost ...  46342.66   
5   2024-01-11  Stablecoin operator Circle files for second at...  46342.66   
6   2024-01-12  SEC’s bitcoin ETF sign-off comes with a stark ...  42773.03   
7   2024-01-18  Firms pitch inverse crypto funds after SEC’s b...  41292.71   
8   2024-01-22  ​SEC says false bitcoin post came after phone ...  39524.27   
9   2024-02-22  The SEC approved Bitcoin ETFs. The European Ce...  51258.70   
10  2024-03-11    UK regulator to allow crypto-related securities  72110.98   
11  2024-04-05  Terraform Labs and Do Kwon found lia